In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

In [ ]:
from datetime import datetime, timedelta
import pandas as pd

In [ ]:
user_transaction = fs.get_feature_group(
    name="user_transaction_raw",
    version=1
)

In [ ]:
df = user_transaction.read(online=True)

In [ ]:
current_datetime = datetime.now()
one_day_ago = datetime.now() - timedelta(days=1)

df = df.drop(['unique_id'], axis=1)[df['created_at'] >= one_day_ago].reset_index()

device_df = df.groupby('device_id')['user_id'].nunique().reset_index().rename(columns={'user_id': 'count'})

final_df = pd.merge(df, device_df, on='device_id').groupby('user_id').agg({'count': 'max'}).reset_index().rename(columns={'count': 'num_users_share_device_id'})

final_df['created_at'] = current_datetime

final_df.head()

In [ ]:
num_users_share_device_id = fs.get_or_create_feature_group(
    name="num_users_share_device_id",
    version=1,
    description="Number of users sharing a device id",
    primary_key=['user_id'],
    event_time='created_at',
    online_enabled=True,
    stream=True
)

num_users_share_device_id.insert(final_df)